In [1]:
import sys
import os 
import nest_asyncio

# Sanity check
print(sys.executable)
nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "sk-"
#os.environ[
#    "AZURE_OPENAI_ENDPOINT"
#] = "https://<your-resource-name>.openai.azure.com/"
#os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

/home/amor/Documents/code_dw/human_in_the_loop_workflow_demo/venv/bin/python3.11


In [2]:
import os
from pydantic import BaseModel, Field
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer



## 1 - Evaluation of correctness 

Use an evaluator and score a dataset of generation


In [3]:


query = (
    "Can you explain the theory of relativity proposed by Albert Einstein in"
    " detail?"
)

reference = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

General relativity, published in 1915, extended these ideas to include the effects of gravity. 
According to general relativity, gravity is not a force between masses, as described by Newton's theory of gravity, 
but rather the result of the warping of space and time by mass and energy. 
Massive objects, such as planets and stars, cause a curvature in spacetime, and smaller objects follow curved paths in 
response to this curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet, 
causing it to create a depression that other objects (representing smaller masses) naturally move towards.

In essence, general relativity provided a new understanding of gravity, explaining phenomena like the bending 
of light by gravity (gravitational lensing) and the precession of the orbit of Mercury. It has been confirmed 
through numerous experiments and observations and has become a fundamental theory in modern physics.
"""

response = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. 
Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating 
observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. 
It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

However, general relativity, published in 1915, extended these ideas to include the effects of magnetism. 
According to general relativity, gravity is not a force between masses but rather the result of the warping of s
pace and time by magnetic fields generated by massive objects. Massive objects, such as planets and stars, 
create magnetic fields that cause a curvature in spacetime, and smaller objects follow curved paths in response 
to this magnetic curvature. This concept is often illustrated using the analogy of a heavy ball placed on a 
rubber sheet with magnets underneath, causing it to create a depression that other objects (representing smaller masses) 
naturally move towards due to magnetic attraction.
"""


In [5]:
from llama_index.core.evaluation import CorrectnessEvaluator
from llama_index.llms.openai import OpenAI

llm = OpenAI("gpt-4o-mini")
evaluator = CorrectnessEvaluator(llm=llm)

result = evaluator.evaluate(
    query=query,
    response=response,
    reference=reference,
)

In [12]:
print(result.score)
print(result.feedback)

2.0
The generated answer is relevant to the user query as it discusses the theory of relativity and its components. However, it contains significant inaccuracies, particularly in the explanation of general relativity, where it incorrectly attributes the warping of spacetime to magnetic fields instead of mass and energy. This fundamental mistake affects the correctness of the answer.


## 2 - RAG faithfulness 

RAG verification after summarization



In [18]:
import os
from pydantic import BaseModel, Field
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import FaithfulnessEvaluator


documents = SimpleDirectoryReader(
    input_files=["./paul_graham_essay.txt"],
).load_data()
splitter = SentenceSplitter(chunk_size=256)
nodes = splitter.get_nodes_from_documents(documents)
retriever_top_5 = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=5,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)


evaluator_gpt4 = FaithfulnessEvaluator(llm=llm)

In [19]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever_top_5,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query("Why did microcomputer emerge ?")

eval_result = evaluator_gpt4.evaluate_response(response=response)

eval_result

EvaluationResult(query=None, contexts=['That\'s not always why artists have a signature style, but it\'s usually why buyers pay a lot for such work. [6]\n\nThere were plenty of earnest students too: kids who "could draw" in high school, and now had come to what was supposed to be the best art school in the country, to learn to draw even better. They tended to be confused and demoralized by what they found at RISD, but they kept going, because painting was what they did. I was not one of the kids who could draw in high school, but at RISD I was definitely closer to their tribe than the tribe of signature style seekers.\n\nI learned a lot in the color class I took at RISD, but otherwise I was basically teaching myself to paint, and I could do that for free. So in 1993 I dropped out. I hung around Providence for a bit, and then my college friend Nancy Parmet did me a big favor. A rent-controlled apartment in a building her mother owned in New York was becoming vacant. Did I want it?', "Sh

In [23]:
print(eval_result.feedback, eval_result.score)

print(response.response)

NO 0.0
To make a shepherd's pie, you typically start by browning ground meat (traditionally lamb, but beef or a mix of both can be used) with onions and garlic. Then, you add vegetables like carrots, peas, and corn, along with seasonings like salt, pepper, and herbs. The mixture is then placed in a baking dish and topped with a layer of mashed potatoes before being baked until the top is golden brown.


In [22]:
response = query_engine.query("How to make a sherpherd pie ?")

eval_result = evaluator_gpt4.evaluate_response(response=response)

print(eval_result.feedback, eval_result.score)
print(response.response)

('NO', 0.0)